<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Keys" data-toc-modified-id="Load-Keys-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Keys</a></span></li><li><span><a href="#Get-Data" data-toc-modified-id="Get-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get Data</a></span></li><li><span><a href="#Borough-Assignment" data-toc-modified-id="Borough-Assignment-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Borough Assignment</a></span><ul class="toc-item"><li><span><a href="#Borough-Assignment-by-Road-Type" data-toc-modified-id="Borough-Assignment-by-Road-Type-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Borough Assignment by Road Type</a></span></li></ul></li><li><span><a href="#Population-Data" data-toc-modified-id="Population-Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Population Data</a></span></li><li><span><a href="#Cleaned-Final-Data" data-toc-modified-id="Cleaned-Final-Data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Cleaned Final Data</a></span><ul class="toc-item"><li><span><a href="#Write-to-CSV" data-toc-modified-id="Write-to-CSV-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Write to CSV</a></span></li></ul></li></ul></div>

# Cyclist and Pedestrain Deaths NYC

Collecting pedestrain and cyclist death by motor vehicle collision data via the NYC Open Data API. Not all boroughs have been properly assigned so we will attempt to assign as many as we can. Once the data is in a good place, we can build out an interactive dashboard in Bokeh to visualize.

## Load Keys

In [1]:
import json
with open('keys.json') as f:
    keys = json.load(f)
    nyc_od_token = keys['nycOD']

## Get Data

First we need to access the NYC Open Data API via an App Token. We can then easily pull in the data on motor vehicle incidents with pedestrains and cyclists. 

In [2]:
# make sure to install these packages before running:
# pip install sodapy
# pip install reverse_geocoder
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import pandas as pd
import os
import re
from sodapy import Socrata

In [3]:
# Go to NYC Open Data website and create an App token to access the aPI
client = Socrata("data.cityofnewyork.us",
                app_token=nyc_od_token)

Socrata has a nice functionality where users can simply query, SQL style, to extract information. It seems like it only accepts basic SQL commands so we will pull down a few columns and use Pandas for the analysis. 

In [4]:
peds_query = """
SELECT  
    borough, crash_date, location, 
    longitude, latitude, on_street_name, 
    off_street_name,cross_street_name,
    number_of_pedestrians_killed, number_of_cyclist_killed,
    contributing_factor_vehicle_1, vehicle_type_code1
WHERE number_of_pedestrians_killed > 0
   OR number_of_cyclist_killed > 0
LIMIT 2000
"""


In [5]:
# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("h9gi-nx95",
                    query=peds_query)

In [6]:
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [7]:
results_df.shape

(1198, 12)

In [8]:
results_df['year'] = pd.DatetimeIndex(results_df['crash_date']).year
results_df['month'] = pd.DatetimeIndex(results_df['crash_date']).month
results_df['day'] = pd.DatetimeIndex(results_df['crash_date']).dayofweek
results_df['date'] = pd.DatetimeIndex(results_df['crash_date']).date


In [9]:
results_df.year.value_counts(sort=False)

2012     78
2013    184
2014    153
2015    146
2016    166
2017    146
2018    130
2019    159
2020     36
Name: year, dtype: int64

In [10]:
results_df.borough.value_counts(dropna=False)

NaN              346
BROOKLYN         281
QUEENS           241
MANHATTAN        192
BRONX            107
STATEN ISLAND     31
Name: borough, dtype: int64

In [11]:
results_df = results_df[['borough','year','month','day', 'date',
                         'location', 'latitude', 'longitude',
                         'cross_street_name','off_street_name', 
                         'on_street_name', 'vehicle_type_code1', 
                         'contributing_factor_vehicle_1', 
                         'number_of_cyclist_killed', 
                         'number_of_pedestrians_killed']]

## Borough Assignment

Many incidents do not have borough assignments, but we want to the know borough assignment for downstream analysis. The data have longitude, latitude values that we can geo-locate, but not all cases have location data. 

In [12]:
import reverse_geocoder as rg

In [13]:
def make_borough_gps(row):
    dict_idx = 0 # one OrderedDict in a list
    borough = 4 # Fourth key is borough
    name = 1 # value index for key/value pair
    cord = (row['coordinates'][1],row['coordinates'][0])
    location = rg.search(cord)
    label = list(location[dict_idx].items())[borough][name]
    if label == 'Kings County': 
        return 'BROOKLYN'
    if label == 'Queens County':
        return 'QUEENS'
    if label == 'New York County':
        return 'MANHATTAN'
    if label == 'Bronx':
        return 'BRONX'
    if label == 'Richmond County':
        return 'STATEN ISLAND'
    else: label = 'NOT NYC'
    return label

In [14]:
results_df['borough_gps'] = results_df['location'].dropna().apply(lambda x: make_borough_gps(x))

Loading formatted geocoded file...


In [15]:
results_df.borough_gps.value_counts(dropna=False)

BROOKLYN         295
QUEENS           286
NOT NYC          188
MANHATTAN        163
NaN              145
BRONX             96
STATEN ISLAND     25
Name: borough_gps, dtype: int64

In [16]:
results_df[(results_df['borough'] != results_df['borough_gps']) & 
           (results_df['borough_gps'].isna() != True) & 
           (results_df['borough'].isna() == True)][['borough', 'borough_gps']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 236 entries, 0 to 1187
Data columns (total 2 columns):
borough        0 non-null object
borough_gps    236 non-null object
dtypes: object(2)
memory usage: 5.5+ KB


There are 236 places where we can fill by the GPS location uncontested. There are more unknowns to fill though since there are many incidents where the GPS location differs from the reported borough. There also places where no location data is present. 

In [17]:
results_df[(results_df['borough'] != results_df['borough_gps']) & 
           (results_df['borough_gps'].isna() != True) &
           (results_df['borough'].isna() != True)][['borough', 'borough_gps']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 249 entries, 9 to 1197
Data columns (total 2 columns):
borough        249 non-null object
borough_gps    249 non-null object
dtypes: object(2)
memory usage: 5.8+ KB


249 incidents where the GPS location differs from the reported borough. We will keep the reported borough as the correct assignment. It should be noted that the police do not always report incidents and locations correctly, but for this analysis we will keep those assignments. 

In [18]:
results_df.loc[results_df.borough.isna() == True, 'borough'] = \
    results_df.loc[results_df.borough.isna() == True, 'borough_gps']

In [19]:
results_df.borough.value_counts(dropna=False)

BROOKLYN         333
QUEENS           308
MANHATTAN        236
BRONX            138
NaN              110
STATEN ISLAND     38
NOT NYC           35
Name: borough, dtype: int64

145 cases where there is no GPS or borough information. These cases could be decided by street names if those are present. We are down to the by-hand work. The geo-locator seems to have only medium reliable so we will keep it in here and potentially reassign borough based on street. 

### Borough Assignment by Road Type

We will now filter down the dataframe by street columns to determine borough assignment. This is mainly to try and break this down into small pieces for assignment. 

In [20]:
missing_boro_df = results_df.loc[(results_df.borough.isna() == True) |
                                 (results_df.borough == 'NOT NYC'), :] \
                                [['location', 'on_street_name', 'off_street_name', 
                                  'cross_street_name', 'borough', 'borough_gps', 
                                  'latitude', 'longitude']]

In [21]:
missing_boro_df[missing_boro_df.on_street_name.str.contains('STREET', regex=True, na=False)].head()

,location,on_street_name,off_street_name,cross_street_name,borough,borough_gps,latitude,longitude
5,"{'type': 'Point', 'coordinates': [-73.98257, 4...",EAST 14 STREET,NaN,NaN,NOT NYC,NOT NYC,40.7313600,-73.9825700
102,"{'type': 'Point', 'coordinates': [-73.988205, ...",PEARL STREET,NaN,NaN,NOT NYC,NOT NYC,40.6922650,-73.9882050
212,"{'type': 'Point', 'coordinates': [-73.99266, 4...",EAST 16 STREET,NaN,NaN,NOT NYC,NOT NYC,40.7373280,-73.9926600
262,"{'type': 'Point', 'coordinates': [-73.964966, ...",WEST 116 STREET,NaN,NaN,NOT NYC,NOT NYC,40.8084830,-73.9649660
277,NaN,BAY STREET,VICTORY BOULEVARD,NaN,NaN,NaN,NaN,NaN


In [22]:
def make_boro_assignment(df, index, borough):
    df.loc[index:index+1, 'borough'] = borough
    return df

In [23]:
boros = ['MANHATTAN', "BROOKLYN", "QUEENS", "STATEN ISLAND", "BRONX"]
results_df = make_boro_assignment(results_df, 176, boros[0])
results_df = make_boro_assignment(results_df, 207, boros[1])
results_df = make_boro_assignment(results_df, 716, boros[0])
results_df = make_boro_assignment(results_df, 493, boros[0])
results_df = make_boro_assignment(results_df, 723, boros[3])
results_df = make_boro_assignment(results_df, 776, boros[2])
results_df = make_boro_assignment(results_df, 816, boros[2])
results_df = make_boro_assignment(results_df, 873, boros[1])
results_df = make_boro_assignment(results_df, 969, boros[2])
results_df = make_boro_assignment(results_df, 1036, boros[1])
results_df = make_boro_assignment(results_df, 1065, boros[1])
results_df = make_boro_assignment(results_df, 1177, boros[3])

In [24]:
avenue_df = results_df.loc[(results_df.borough.isna() == True) |
                            (results_df.borough == 'NOT NYC'), :] \
                            [['on_street_name', 'off_street_name', 
                              'cross_street_name', 'borough', 'borough_gps']]
avenue_df[(avenue_df.on_street_name.str.contains('AVENUE',regex=True, na=False))].head()

,on_street_name,off_street_name,cross_street_name,borough,borough_gps
36,BORDEN AVENUE,58 ROAD,NaN,NaN,NaN
46,JAMAICA AVENUE,80 STREET,NaN,NaN,NaN
216,11 AVENUE,NaN,NaN,NOT NYC,NOT NYC
241,AVENUE OF THE AMERICAS,WEST HOUSTON STREET,NaN,NaN,NaN
332,10 AVENUE,NaN,NaN,NOT NYC,NOT NYC


In [25]:
results_df = make_boro_assignment(results_df, 27, boros[2])
results_df = make_boro_assignment(results_df, 304, boros[0])
results_df = make_boro_assignment(results_df, 399, boros[0])
results_df = make_boro_assignment(results_df, 619, boros[4])
results_df = make_boro_assignment(results_df, 637, boros[4])
results_df = make_boro_assignment(results_df, 663, boros[1])
results_df = make_boro_assignment(results_df, 673, boros[1])
results_df = make_boro_assignment(results_df, 678, boros[3])
results_df = make_boro_assignment(results_df, 739, boros[1])
results_df = make_boro_assignment(results_df, 751, boros[2])
results_df = make_boro_assignment(results_df, 755, boros[2])
results_df = make_boro_assignment(results_df, 757, boros[2])
results_df = make_boro_assignment(results_df, 764, boros[4])
results_df = make_boro_assignment(results_df, 785, boros[1])
results_df = make_boro_assignment(results_df, 790, boros[1])
results_df = make_boro_assignment(results_df, 835, boros[2])
results_df = make_boro_assignment(results_df, 891, boros[2])
results_df = make_boro_assignment(results_df, 895, boros[1])
results_df = make_boro_assignment(results_df, 914, boros[1])
results_df = make_boro_assignment(results_df, 915, boros[2])
results_df = make_boro_assignment(results_df, 953, boros[2])
results_df = make_boro_assignment(results_df, 961, boros[4])

In [26]:
avenue_df = results_df.loc[(results_df.borough.isna() == True) |
                            (results_df.borough == 'NOT NYC'), :] \
                            [['on_street_name', 'off_street_name', 
                              'cross_street_name', 'borough', 'borough_gps']]
avenue_df[avenue_df.on_street_name.str.contains('AVENUE',regex=True, na=False)].head()

,on_street_name,off_street_name,cross_street_name,borough,borough_gps
36,BORDEN AVENUE,58 ROAD,NaN,NaN,NaN
46,JAMAICA AVENUE,80 STREET,NaN,NaN,NaN
216,11 AVENUE,NaN,NaN,NOT NYC,NOT NYC
241,AVENUE OF THE AMERICAS,WEST HOUSTON STREET,NaN,NaN,NaN
332,10 AVENUE,NaN,NaN,NOT NYC,NOT NYC


In [27]:
results_df = make_boro_assignment(results_df, 959, boros[2])
results_df = make_boro_assignment(results_df, 975, boros[1])
results_df = make_boro_assignment(results_df, 1004, boros[2])
results_df = make_boro_assignment(results_df, 1005, boros[4])
results_df = make_boro_assignment(results_df, 1028, boros[0])
results_df = make_boro_assignment(results_df, 1031, boros[1])
results_df = make_boro_assignment(results_df, 1039, boros[1])
results_df = make_boro_assignment(results_df, 1070, boros[4])
results_df = make_boro_assignment(results_df, 1083, boros[1])
results_df = make_boro_assignment(results_df, 1086, boros[2])
results_df = make_boro_assignment(results_df, 1087, boros[2])
results_df = make_boro_assignment(results_df, 1174, boros[0])

In [28]:
blvd_df = results_df.loc[(results_df.borough.isna() == True) |
              (results_df.borough == 'NOT NYC'), :] \
              [['location', 'on_street_name', 'off_street_name', 
              'cross_street_name', 'borough', 'borough_gps', 
              'latitude', 'longitude']]
blvd_df[blvd_df.on_street_name.str.contains('BOULEVARD|SQUARE', regex=True, na=False)].head()

,location,on_street_name,off_street_name,cross_street_name,borough,borough_gps,latitude,longitude
133,NaN,HIGHLAND BOULEVARD,VERMONT STREET,NaN,NaN,NaN,NaN,NaN
137,NaN,ROCKAWAY BOULEVARD,NaN,NaN,NaN,NaN,NaN,NaN
141,NaN,CONDUIT BOULEVARD,CRESCENT STREET,NaN,NaN,NaN,NaN,NaN
194,NaN,CROSSBAY BOULEVARD,149 AVENUE,NaN,NaN,NaN,NaN,NaN
326,"{'type': 'Point', 'coordinates': [-73.824684, ...",PARSONS BOULEVARD,NaN,NaN,NOT NYC,NOT NYC,40.7730600,-73.8246840


In [29]:
# boros = ['MANHATTAN', "BROOKLYN", "QUEENS", "STATEN ISLAND", "BRONX"]
results_df = make_boro_assignment(results_df, 82, boros[2])
results_df = make_boro_assignment(results_df, 85, boros[1])
results_df = make_boro_assignment(results_df, 223, boros[2])
results_df = make_boro_assignment(results_df, 347, boros[2])
results_df = make_boro_assignment(results_df, 453, boros[4])
results_df = make_boro_assignment(results_df, 540, boros[2])
results_df = make_boro_assignment(results_df, 553, boros[2])
results_df = make_boro_assignment(results_df, 567, boros[2])
results_df = make_boro_assignment(results_df, 644, boros[2])
results_df = make_boro_assignment(results_df, 658, boros[2])
results_df = make_boro_assignment(results_df, 680, boros[2])
results_df = make_boro_assignment(results_df, 801, boros[1])
results_df = make_boro_assignment(results_df, 833, boros[2])
results_df = make_boro_assignment(results_df, 906, boros[0])
results_df = make_boro_assignment(results_df, 917, boros[4])
results_df = make_boro_assignment(results_df, 932, boros[2])
results_df = make_boro_assignment(results_df, 1077, boros[2])
results_df = make_boro_assignment(results_df, 1082, boros[3])
results_df = make_boro_assignment(results_df, 1156, boros[1])
results_df = make_boro_assignment(results_df, 1163, boros[2])

In [30]:
off_avenue = results_df.loc[(results_df.borough.isna() == True) |
              (results_df.borough == 'NOT NYC'), :] \
              [['location', 'on_street_name', 'off_street_name', 
              'cross_street_name', 'borough', 'borough_gps']]
off_avenue[off_avenue.off_street_name.str.contains('AVENUE', regex=True, na=False)]

,location,on_street_name,off_street_name,cross_street_name,borough,borough_gps
194,NaN,CROSSBAY BOULEVARD,149 AVENUE,NaN,NaN,NaN
788,NaN,WESTCHESTER AVENUE,EAST TREMONT AVENUE,NaN,NaN,NaN
885,NaN,165 STREET,HILLSIDE AVENUE,NaN,NaN,NaN
910,NaN,FLATBUSH AVENUE,DEKALB AVENUE,NaN,NaN,NaN
921,NaN,80 STREET,COOPER AVENUE,NaN,NaN,NaN
1018,NaN,UNION AVENUE,MEEKER AVENUE,NaN,NaN,NaN
1049,NaN,ASTORIA BOULEVARD,31 AVENUE,NaN,NaN,NaN
1057,NaN,DRUMGOOLE ROAD EAST,WOLCOTT AVENUE,NaN,NaN,NaN
1059,NaN,EASTERN PARKWAY,SAINT MARKS AVENUE,NaN,NaN,NaN
1069,NaN,WEBSTER AVENUE,CLAY AVENUE,NaN,NaN,NaN


In [31]:
results_df = make_boro_assignment(results_df, 937, boros[1])
results_df = make_boro_assignment(results_df, 939, boros[2])
results_df = make_boro_assignment(results_df, 949, boros[3])
results_df = make_boro_assignment(results_df, 968, boros[1])
results_df = make_boro_assignment(results_df, 990, boros[3])
results_df = make_boro_assignment(results_df, 1050, boros[4])

In [32]:
results_df = make_boro_assignment(results_df, 73, boros[2])
results_df = make_boro_assignment(results_df, 148, boros[2])
results_df = make_boro_assignment(results_df, 234, boros[4])
results_df = make_boro_assignment(results_df, 300, boros[2])
results_df = make_boro_assignment(results_df, 308, boros[2])
results_df = make_boro_assignment(results_df, 397, boros[2])
results_df = make_boro_assignment(results_df, 401, boros[1])
results_df = make_boro_assignment(results_df, 405, boros[0])
results_df = make_boro_assignment(results_df, 468, boros[2])
results_df = make_boro_assignment(results_df, 589, boros[0])

In [33]:
results_df = make_boro_assignment(results_df, 546, boros[3])
results_df = make_boro_assignment(results_df, 558, boros[2])
results_df = make_boro_assignment(results_df, 591, boros[4])
results_df = make_boro_assignment(results_df, 574, boros[2])
results_df = make_boro_assignment(results_df, 594, boros[0])
results_df = make_boro_assignment(results_df, 604, boros[4])
results_df = make_boro_assignment(results_df, 628, boros[4])
results_df = make_boro_assignment(results_df, 697, boros[3])
results_df = make_boro_assignment(results_df, 762, boros[4])
results_df = make_boro_assignment(results_df, 822, boros[4])
results_df = make_boro_assignment(results_df, 850, boros[1])
results_df = make_boro_assignment(results_df, 863, boros[2])
results_df = make_boro_assignment(results_df, 969, boros[3])
results_df = make_boro_assignment(results_df, 1009, boros[2])
results_df = make_boro_assignment(results_df, 1038, boros[1])
results_df = make_boro_assignment(results_df, 1041, boros[2])
results_df = make_boro_assignment(results_df, 1138, boros[1])

## Population Data

QUEENS

Year 
        Population
                    Growth
                            Growth Rate
2018	2,278,906	-17,959	-0.78%
2017	2,296,865	-12,167	-0.53%
2016	2,309,032	1,418	0.06%
2015	2,307,614	7,466	0.32%
2014	2,300,148	11,954	0.52%
2013	2,288,194	15,449	0.68%
2012	2,272,745	17,186	0.76%

In [53]:
pop_queens = [
    2272745,
    2288194,
    2300148,
    2307614,
    2309032,
    2296865, 
    2278906,
    2278906
]

Bronx

Year 
        Population
                    Growth
                            Growth Rate
2018	1,432,132	-7,593	-0.53%
2017	1,439,725	-5,512	-0.38%
2016	1,445,237	4,511	0.31%
2015	1,440,726	9,188	0.64%
2014	1,431,538	9,198	0.65%
2013	1,422,340	10,625	0.75%
2012	1,411,715	14,349	1.03%

In [54]:
pop_bronx = [
    1411715,
    1422340,
    1431538,
    1440726,
    1445237,
    1439725, 
    1432132,
    1432132
    ]

manhattan

Year 
        Population
                    Growth
                        Growth Rate
2018	1,628,701	-1,079	-0.07%
2017	1,629,780	-6,481	-0.40%
2016	1,636,261	-452	-0.03%
2015	1,636,713	5,767	0.35%
2014	1,630,946	3,140	0.19%
2013	1,627,806	3,767	0.23%
2012	1,624,039	15,824	0.98%

In [55]:
pop_manhattan = [
    1624039,
    1627806,
    1630946,
    1636713,
    1636261,
    1629780, 
    1628701,
    1628701
    ]

Brooklyn

Year 
        Population
                    Growth
                        Growth Rate
2018	2,582,830	-13,555	-0.52%
2017	2,596,385	-14,847	-0.57%
2016	2,611,232	625	0.02%
2015	2,610,607	7,927	0.30%
2014	2,602,680	14,179	0.55%
2013	2,588,501	19,625	0.76%
2012	2,568,876	28,054	1.10%

In [56]:
pop_brooklyn = [
    2568876,
    2588501,
    2602680,
    2610607,
    2611232,
    2596385, 
    2582830,
    2582830
]

Staten Island

Year 
        Population
                Growth
                        Growth Rate
2018	476,179	663	0.14%
2017	475,516	1,302	0.27%
2016	474,214	1,693	0.36%
2015	472,521	446	0.09%
2014	472,075	177	0.04%
2013	471,898	1,241	0.26%
2012	470,657	-344	-0.07%

In [57]:
pop_si_ls = [
    470657,
    471898,
    472075,
    472521,
    474214,
    475516, 
    476179, 
    476179
]
years_ls = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
pop_lists = [pop_queens, pop_brooklyn, pop_manhattan, pop_si_ls, pop_bronx]
borough_ls = results_df.borough.unique().tolist()
borough_ls.pop(1)
borough_ls.pop(2)
borough_ls


['QUEENS', 'BROOKLYN', 'MANHATTAN', 'STATEN ISLAND', 'BRONX']

In [58]:
pop_df = pd.DataFrame(columns=['year', 'population'])
for i in range(5):
    boro_pop = pd.DataFrame(list(zip(years_ls, pop_lists[i])), columns=['year', 'population'])
    boro_pop['borough'] = borough_ls[i]
    pop_df = pd.concat([pop_df, boro_pop], ignore_index=True, sort=True)

## Cleaned Final Data

The remaining data that do not have borough assignments are locations that could be in one of several boroughs or is a collision that occured on an interstate highway such as I-87, I-495, I-278, etc. 

In [40]:
results_df.head()

,borough,year,month,day,date,location,latitude,longitude,cross_street_name,off_street_name,on_street_name,vehicle_type_code1,contributing_factor_vehicle_1,number_of_cyclist_killed,number_of_pedestrians_killed,borough_gps
0,QUEENS,2013,2,5,2013-02-23,"{'type': 'Point', 'coordinates': [-73.9544246,...",40.7624116,-73.9544246,NaN,NaN,NaN,PASSENGER VEHICLE,Unspecified,0,1,QUEENS
1,NaN,2013,2,4,2013-02-15,NaN,NaN,NaN,NaN,NaN,NaN,PASSENGER VEHICLE,Unspecified,0,1,NaN
2,QUEENS,2020,4,1,2020-04-14,"{'type': 'Point', 'coordinates': [-73.83301, 4...",40.7478330,-73.8330100,NaN,NaN,COLLEGE POINT BOULEVARD,Station Wagon/Sport Utility Vehicle,Traffic Control Disregarded,0,1,QUEENS
3,QUEENS,2020,5,0,2020-05-18,"{'type': 'Point', 'coordinates': [-73.827286, ...",40.7048570,-73.8272860,124-50 METROPOLITAN AVENUE,NaN,NaN,Box Truck,View Obstructed/Limited,0,1,QUEENS
4,BROOKLYN,2020,4,1,2020-04-28,"{'type': 'Point', 'coordinates': [-73.95166, 4...",40.6430630,-73.9516600,NaN,CLARENDON ROAD,ROGERS AVENUE,Bus,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,1,0,BROOKLYN


In [41]:
results_df.borough.value_counts(dropna=False)

QUEENS           348
BROOKLYN         316
MANHATTAN        225
BRONX            146
NaN               84
STATEN ISLAND     50
NOT NYC           29
Name: borough, dtype: int64

Once most of the unassigned deaths were manually reassigned or reassigned by geo-location, Queens has moved into unfortunate position of most deaths for pedestrains and cyclists. 

In [42]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198 entries, 0 to 1197
Data columns (total 16 columns):
borough                          1114 non-null object
year                             1198 non-null int64
month                            1198 non-null int64
day                              1198 non-null int64
date                             1198 non-null object
location                         1053 non-null object
latitude                         1053 non-null object
longitude                        1053 non-null object
cross_street_name                77 non-null object
off_street_name                  910 non-null object
on_street_name                   1082 non-null object
vehicle_type_code1               1182 non-null object
contributing_factor_vehicle_1    1187 non-null object
number_of_cyclist_killed         1198 non-null object
number_of_pedestrians_killed     1198 non-null object
borough_gps                      1053 non-null object
dtypes: int64(3), object(13)
memory

In [43]:
def apply_row_regex(df_col, old_str, replacement_str):
    df_col.replace(to_replace=old_str,
                    value= replacement_str,
                    regex=True, 
                    inplace=True)
    return df_col

In [44]:
apply_row_regex(results_df.contributing_factor_vehicle_1,
                old_str="Driver Inattention/Distraction",
                replacement_str="Driver Distraction")
apply_row_regex(results_df.contributing_factor_vehicle_1,
                old_str="Failure to Yield Right-of-Way",
                replacement_str="Failure to Yield")
apply_row_regex(results_df.contributing_factor_vehicle_1,
                old_str="Traffic Control Disregarded",
                replacement_str="Traffic Control Ignored")
apply_row_regex(results_df.contributing_factor_vehicle_1,
                old_str="Aggressive Driving/Road Rage",
                replacement_str="Aggressive Driving")
apply_row_regex(results_df.contributing_factor_vehicle_1,
                old_str="Passing or Lane Usage Improper",
                replacement_str="Lane Usage Improper")
apply_row_regex(results_df.contributing_factor_vehicle_1,
                old_str="Driverless/Runaway Vehicle",
                replacement_str="Driverless Vehicle")
apply_row_regex(results_df.contributing_factor_vehicle_1,
                old_str="Reaction to Uninvolved Vehicle",
                replacement_str="Reacted to Other Vehicle")
apply_row_regex(results_df.contributing_factor_vehicle_1,
                old_str="Pedestrian/Bicyclist/Other Pedestrian Error/Confusion",
                replacement_str="Pedestrian/Cyclist Error")
apply_row_regex(results_df.contributing_factor_vehicle_1,
                old_str="Drugs \(illegal\)",
                replacement_str="Drugs (Illegal)")

0                    Unspecified
1                    Unspecified
2        Traffic Control Ignored
3        View Obstructed/Limited
4       Pedestrian/Cyclist Error
5               Backing Unsafely
6                    Unspecified
7                    Unspecified
8             Driver Distraction
9            Alcohol Involvement
10       Traffic Control Ignored
11              Failure to Yield
12                   Unspecified
13              Failure to Yield
14       View Obstructed/Limited
15                   Unspecified
16              Failure to Yield
17                  Unsafe Speed
18                  Unsafe Speed
19            Driver Distraction
20                   Unspecified
21            Driver Distraction
22            Driver Distraction
23              Backing Unsafely
24                   Unspecified
25              Failure to Yield
26              Failure to Yield
27                   Unspecified
28            Driver Distraction
29      Pedestrian/Cyclist Error
          

In [45]:
results_df.vehicle_type_code1 = results_df.vehicle_type_code1 \
                 .replace(np.nan, 'NaN', regex=True)

In [46]:
results_df.vehicle_type_code1 = results_df.vehicle_type_code1 \
                 .apply(lambda x: x.title())

In [47]:
apply_row_regex(results_df.vehicle_type_code1,
                old_str="Sport Utility / Station Wagon",
                replacement_str="SUV / Station Wagon")
apply_row_regex(results_df.vehicle_type_code1,
                old_str="Station Wagon/Sport Utility Vehicle",
                replacement_str="SUV / Station Wagon")
apply_row_regex(results_df.vehicle_type_code1,
                old_str='Large Com Veh\(6 Or More Tires\)',
                replacement_str="Com Veh, 6+ Tires")
apply_row_regex(results_df.vehicle_type_code1,
                old_str='Small Com Veh\(4 Tires\)',
                replacement_str="Com Veh, 4 Tires")
apply_row_regex(results_df.vehicle_type_code1,
                old_str="Tractor Truck Gasoline",
                replacement_str="Tractor Truck")
apply_row_regex(results_df.vehicle_type_code1,
                old_str="Tractor Truck Diesel",
                replacement_str="Tractor Truck")

0         Passenger Vehicle
1         Passenger Vehicle
2       SUV / Station Wagon
3                 Box Truck
4                       Bus
5                      Taxi
6                     Sedan
7             Tractor Truck
8       SUV / Station Wagon
9       SUV / Station Wagon
10               Motorcycle
11            Pick-Up Truck
12                    E-Sco
13      SUV / Station Wagon
14                     Dump
15                    Sedan
16      SUV / Station Wagon
17                      Nan
18                    Sedan
19                      Van
20                      Bus
21                    Sedan
22                      Bus
23      SUV / Station Wagon
24      SUV / Station Wagon
25      SUV / Station Wagon
26                    Sedan
27      SUV / Station Wagon
28           Concrete Mixer
29                      Bus
               ...         
1168      Passenger Vehicle
1169    SUV / Station Wagon
1170      Passenger Vehicle
1171      Passenger Vehicle
1172      Passenger 

### Write to CSV

In [60]:
#results_df.to_csv('peds_death_data')
#pop_df.to_csv('pop_borough')

Now that we have the data in a good place we can build out an interactive dashboard. 